In [6]:
!pip install --upgrade torch
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,LabelEncoder
from torch.utils.data import DataLoader , Dataset

In [49]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")


In [32]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [34]:
train_data['Age'].fillna(train_data['Age'].median(), inplace = True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0] , inplace = True)
train_data['Fare'].fillna(train_data['Fare'].median() , inplace= True)

<ipython-input-34-39fda68e03cb>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(), inplace = True)
<ipython-input-34-39fda68e03cb>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inpl

In [35]:
train_data.drop(['PassengerId','Name','Ticket','Cabin'] , axis = 1,inplace=True)

In [36]:
label_encoder = LabelEncoder()
train_data['Sex'] = label_encoder.fit_transform(train_data['Sex'])
train_data['Embarked'] = label_encoder.fit_transform(train_data['Embarked'])

In [37]:
X = train_data.drop('Survived',axis= 1).values
y = train_data['Survived'].values

In [38]:
X_train,X_val,y_train,y_val = train_test_split(X ,y , test_size = 0.2 , random_state = 42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [39]:
class TitanicDataSet(Dataset):
  def __init__(self,features,labels):
    self.features =torch.tensor(features , dtype = torch.float32)
    self.labels = torch.tensor(labels , dtype= torch.float32)

  def __len__(self):
    return len(self.features)

  def __getitem__(self , idx):
    return self.features[idx] , self.labels[idx]


train_datasets = TitanicDataSet(X_train,y_train)
val_datasets = TitanicDataSet(X_val , y_val)

train_loader = DataLoader(train_datasets,batch_size = 32 , shuffle = True)
val_loader = DataLoader(val_datasets,batch_size = 32 , shuffle=False)

In [58]:
class TitanicModel(nn.Module):
  def __init__(self):
    super(TitanicModel,self).__init__()

    self.fc1 = nn.Linear(7 , 64)
    self.relu1 = nn.ReLU()
    self.fc4 = nn.Linear(64,128)
    self.relu4 =nn.ReLU()
    self.fc5 = nn.Linear(128,64)
    self.relu5 =nn.ReLU()
    self.fc2 = nn.Linear(64,32)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(32,1)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    x = self.relu1(self.fc1(x))
    x = self.relu2(self.fc2(x))
    x = self.sigmoid(self.fc3(x))
    return x

model = TitanicModel()



In [59]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters() , lr = 0.001 )

In [60]:
num_epochs = 100
for epoch in range(num_epochs):
  model.train()
  train_loss = 0.0
  for features , labels in train_loader:
    outputs = model(features.float()).squeeze()
    loss = criterion(outputs , labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    train_loss += loss.item()

  model.eval()
  val_loss = 0.0
  with torch.no_grad():
    for features ,labels in val_loader:
      outputs = model(features).squeeze()
      loss = criterion(outputs ,labels)
      val_loss += loss.item()

  print(f"Epochs {epoch+1}/{num_epochs} , Train_loss: {train_loss/len(train_loader):.4f} , value loss: {val_loss/len(val_loader):.4f}")


Epochs 1/100 , Train_loss: 0.6676 , value loss: 0.6192
Epochs 2/100 , Train_loss: 0.5811 , value loss: 0.5256
Epochs 3/100 , Train_loss: 0.4987 , value loss: 0.4621
Epochs 4/100 , Train_loss: 0.4545 , value loss: 0.4347
Epochs 5/100 , Train_loss: 0.4372 , value loss: 0.4227
Epochs 6/100 , Train_loss: 0.4360 , value loss: 0.4211
Epochs 7/100 , Train_loss: 0.4200 , value loss: 0.4180
Epochs 8/100 , Train_loss: 0.4136 , value loss: 0.4153
Epochs 9/100 , Train_loss: 0.4220 , value loss: 0.4207
Epochs 10/100 , Train_loss: 0.4025 , value loss: 0.4221
Epochs 11/100 , Train_loss: 0.4012 , value loss: 0.4207
Epochs 12/100 , Train_loss: 0.4093 , value loss: 0.4204
Epochs 13/100 , Train_loss: 0.4111 , value loss: 0.4245
Epochs 14/100 , Train_loss: 0.3898 , value loss: 0.4249
Epochs 15/100 , Train_loss: 0.4036 , value loss: 0.4295
Epochs 16/100 , Train_loss: 0.3965 , value loss: 0.4286
Epochs 17/100 , Train_loss: 0.3875 , value loss: 0.4319
Epochs 18/100 , Train_loss: 0.3844 , value loss: 0.4292
E

In [61]:
model.eval()
y_pred = []
y_true = []

with torch.no_grad():
  for features, labels in val_loader:
    outputs = model(features).squeeze()
    preds = (outputs > 0.5).float()
    y_pred.extend(preds.numpy())
    y_true.extend(labels.numpy())

accuracy = np.mean(np.array(y_pred) == np.array(y_true))
print(f"Validation Accuracy : {accuracy:.4f}")





Validation Accuracy : 0.8324


In [62]:
test_data.head()




,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2


In [63]:
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Embarked'].fillna(test_data['Embarked'].mode()[0], inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)





label_encoder = LabelEncoder()


train_data['Sex'] = label_encoder.fit_transform(train_data['Sex'])
test_data['Sex'] = label_encoder.transform(test_data['Sex'])  # Use transform, not fit_transform, for test data

train_data['Embarked'] = label_encoder.fit_transform(train_data['Embarked'])
test_data['Embarked'] = label_encoder.transform(test_data['Embarked'])  # Same here, use transform


<ipython-input-63-00c429bfa707>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
<ipython-input-63-00c429bfa707>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [64]:
X_test = scaler.transform(test_data.values)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
model.eval()
test_preds = model(X_test_tensor).squeeze().detach().numpy()
test_preds = (test_preds > 0.5).astype(int)
submission = pd.read_csv('gender_submission.csv')
submission['Survived'] = test_preds
submission.to_csv('submission.csv', index=False)